In [1]:
import tensorflow as tf 
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.optimizers as optim

import pandas as pd 
import numpy as np 
import os

In [4]:
PATH = r"C:\Users\suyash\Desktop\toxic\tox21_dense_train.csv"
data = pd.read_csv(PATH).values
labels = pd.read_csv(PATH).columns
labels

Index(['Unnamed: 0', 'AW', 'AWeight', 'Arto', 'BertzCT', 'Chi0', 'Chi1',
       'Chi10', 'Chi2', 'Chi3',
       ...
       'W3D', 'W3DH', 'WNSA1', 'WNSA2', 'WNSA3', 'WPSA1', 'WPSA2', 'WPSA3',
       'grav', 'rygr'],
      dtype='object', length=802)

In [32]:
for x in range(799):
    if max(data[:, 3:][:, x])!=min(data[:, 3:][:, x]):
        xtr = (data[:, 3:][:, x]-np.mean(data[:, 3:][:, x]))/(max(data[:, 3:][:, x])-min(data[:, 3:][:, x]))
        data[:, 3:][:, x] = xtr
    else:
        xtr = (data[:, 3:][:, x])/(max(data[:, 3:][:, x])+min(data[:, 3:][:, x])+1e-10)+0.5
        data[:, 3:][:, x] = xtr

In [37]:
data = data[:, 3:]

In [41]:
xtrain = np.array(data[:10000], dtype="float32")
xvalid = np.array(data[10000:12000], dtype="float32")

In [12]:
def DenseBlock(x, ndim):
    x = layers.Dense(ndim)(x)
    x = layers.BatchNormalization(momentum=0.5)(x)
    x = layers.ReLU()(x)
    return x

def autoencoder(inp):
    inp = layers.Input(inp)
    x = DenseBlock()
    enc_out = layers.Dense(32, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(enc_out)
    dec_out = layers.Dense(799, activation="relu")(x)
    encoder = models.Model(inputs=inp, outputs=enc_out, name="Encoder")
    model = models.Model(inputs=inp, outputs=dec_out, name="Autoencoder")

    return encoder, model

encoder, model = autoencoder((799))
print(encoder.summary())
print(model.summary())

Model: "Encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 800)]             0         
                                                                 
 dense_27 (Dense)            (None, 128)               102528    
                                                                 
 dense_28 (Dense)            (None, 32)                4128      
                                                                 
Total params: 106,656
Trainable params: 106,656
Non-trainable params: 0
_________________________________________________________________
None
Model: "Autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 800)]             0         
                                                                 
 dense_27 (Dense)          

In [42]:
model.compile(loss="mse", optimizer="adam", metrics="accuracy")
model.fit(xtrain, xtrain, batch_size=100, verbose=1, epochs=3, validation_data=[xvalid, xvalid], validation_batch_size=100)

Epoch 1/3


ValueError: in user code:

    File "c:\Users\suyash\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\suyash\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\suyash\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\suyash\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\suyash\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\suyash\A